In [1]:
%pip install transformers
import torch
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import BertTokenizer, BertModel
from transformers import logging
logging.set_verbosity_error()

In [3]:
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F

In [4]:
class Mod(torch.nn.Module):
  def __init__(self, num_hidden, num_outs):
    super().__init__()
    
    self.fc1 = torch.nn.Linear(768,num_hidden)
    
    #self.fc2 = torch.nn.Linear(500,num_hidden)
    self.drop = torch.nn.Dropout(0.33)
    self.hidden = torch.nn.Linear(num_hidden,num_outs)

  def forward(self,x):
   # x = self.pool(x)
    x = torch.nn.functional.relu(self.fc1(x))
    #x = torch.nn.Dropout(0.2)(x)
    x = self.drop(x)
   
    #x = torch.nn.functional.relu(self.fc2(x))
    #x = torch.nn.Dropout(0.2)(x)
    return self.hidden(x)

device = "cuda:0" #might need to change this or add an if cuda.is_available() before it
model = Mod(num_hidden = 300, num_outs = 2).to(device) 
#model.load_state_dict(torch.load("model_trained_on_fakeddit.pt"))#,state_dict="state_dict")
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss() #I don't think the paper mentioned loss function used
epochs = 10

#Things to tune would be epochs, learning rate for adam, number of hidden layers
#Haven't implemented use of validation set yet

In [5]:
#train2df = pd.read_csv("train10bd.tsv", delimiter="\t")
test2df = pd.read_csv('test2bd.tsv', delimiter='\t')



In [6]:
from torch import tensor
def converter(x):
    return eval(x)
    

#train2df['embedding'] = train2df['embedding'].apply(converter) #(Takes about 10 min on my pc)





In [7]:
test2df['embedding'] = test2df['embedding'].apply(converter)





In [8]:

#trainset = TensorDataset(torch.stack(list(train2df['embedding'])), torch.LongTensor(list(train2df['label'])))#change depending on 2 3 6 way - 3 and 6 uses _ instead of -
#trainloader = DataLoader(trainset)

testset = TensorDataset(torch.stack(list(test2df['embedding'])), torch.LongTensor(list(test2df['label'])))#change depending on 2 3 6 way
testloader = DataLoader(testset)
print(len(testloader))
import os

model.load_state_dict(torch.load("combined_model.pt"))

400


<All keys matched successfully>

In [17]:
from sklearn import metrics
for epoch in range(epochs):
    successes = 0
    ypred = []
    ytrue = []
    model.train()
    i = 0
    #for data, target in trainloader:
            
           
        #try:
            # optimizer.zero_grad()
            
            # data = data.to(device)
            
            # output = model(data[0].to(device)).to(device)
        
            # loss = criterion(output,target.to(device))
            
            # loss.backward()
            # optimizer.step()
        #except:
           
    
    model.eval()
    with torch.no_grad():
        for i,r in test2df.iterrows():
            
            #try:
                
             
                
                output = model(r["embedding"])
                ypred.append(output.argmax().cpu())
                ytrue.append(r["label"])
            
                
                if output.argmax() == r["label"]:
                    successes += 1
                else:
                    print(r["text"] + "\n label:"+ str(r["label"]) )
                     
            #except:
                
    

    print("EPOCH: " + str(epoch + 1) + " Accuracy:" + str(successes/len(testloader)) )
    print("F1: " + str(metrics.f1_score(ytrue,ypred)))
    print("AUC: " + str(metrics.roc_auc_score(ytrue,ypred)))
    break
    

#Takes around 9 min per epoch on my pc when running the full dataset            




another pardon disgrace
five tips for smart people in a not-so-smart world
elton john denies trump advisor’s claim that he’s performing at inauguration
abusive police technology and dr. turi's predictions
goodwill to pay disabled workers in loose change from donated couches
bank of canada has a new chief
spotted: hillary clinton with bill clinton at a rhode island bookstore - cnnpolitics.com
fedex truck splits in half after crash with passenger train
boyfriend of pregnant md. teacher indicted in her murder - nbc washington
of unicorns and satyrs and things with no knees
bach forever! camerata pacifica enters the fray
india to revamp its submarine fleet to meet admist rising naval tensions in the region
1st skill-based slot machines in us launch in atlantic city
armless pianist begins world tour
forcing a final four not a motivation for ateneo win vs la salle – arespacochaga
the analyst view on ad tech company rubicon project is brutal right now
paki pastor denied bail for blasphemy
gre

In [10]:
#torch.save(model.state_dict(),"combined_model.pt")